In [10]:
#Baseline:SVD，Surprise，testset only have positive sample.
import surprise as sp
import pandas as pd
import numpy as np
import math
from collections import defaultdict

#1.Loading the dataset and Excluding the outliers
kbdata = pd.read_csv("/data/fjsdata/ctKngBase/kb.csv", sep='|', low_memory=False)
kbdata = kbdata.loc[(kbdata['num']<200)]#seven months, one per day
num_max=kbdata['num'].max()
num_min=kbdata['num'].min()
kbdata['num']=kbdata['num'].apply(lambda x: (x-num_min+1)*1.0/(num_max-num_min+1) )
print ('Dataset shape is:%d rows and %d columns'%(kbdata.shape[0],kbdata.shape[1]))

#2.Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(kbdata[['csr', 'ke', 'num']],reader)
# sampling random trainset and testset, and test set is made of 10% of the ratings.
#trainset, testset = sp.model_selection.train_test_split(spdata, test_size=.1)
trainset = spdata.build_full_trainset()
testset = trainset.build_testset()

#3.Training the model and predicting ratings for the testset
algo = sp.SVD()
algo.fit(trainset)
predictions = algo.test(testset)#testset include positive and negtive sample.

#4.measuring the performance of SVD by precision, recall and  NDCG
#print ('RMSE of testset is:%.8f'%(sp.accuracy.rmse(predictions)))
def calc_dcg(items):
    dcg = 0
    i = 0
    for item in items:
        i += 1
        dcg += (math.pow(2, item) - 1)/ math.log(1 + i, 2)
    return dcg
def index_at_k(predictions, k, threshold=0.1):
   #Return precision and recall at k metrics for each user.
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    ndcgs =dict()
    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        # Number of relevant items
        n_rel = sum((true_r > threshold) for (_, true_r) in user_ratings)
        # Number of recommended items in top k
        n_rec_k = sum((est > threshold) for (est, _) in user_ratings[:k])
        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r > threshold) and (est > threshold)) for (est, true_r) in user_ratings[:k])
        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1
        #true ratings of recommended items in top k
        l_rec_k = [true_r for (_,true_r) in user_ratings[:k]]
        dcg = calc_dcg(l_rec_k)
        #l_rec_k.sort(reverse=True)
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        l_rel_k = [true_r for (_,true_r) in user_ratings[:k]]
        idcg = calc_dcg(l_rel_k)
        ndcgs[uid]=dcg*1.0/idcg 
    return precisions, recalls, ndcgs

print ("%3s%20s%20s%20s" % ('K','Precisions','Recalls','NDCG'))
for k in [5,10,15,20]:#latent factor
    precisions, recalls, ndcgs = index_at_k(predictions, k=k)
    # Precision and recall can then be averaged over all users
    precision = sum(prec for prec in precisions.values()) / len(precisions)
    recall = sum(rec for rec in recalls.values()) / len(recalls)
    ndcg = sum(ndcg for ndcg in ndcgs.values()) / len(ndcgs)
    print ("%3s%20.8f%20.8f%20.8f" % (k, precision, recall, ndcg))

Dataset shape is:2547452 rows and 3 columns
  K          Precisions             Recalls                NDCG
  5          0.46939605          0.56137439          0.60653518
 10          0.45245813          0.61192698          0.65193114
 15          0.44844596          0.62378710          0.67542627
 20          0.44742766          0.62568932          0.69187443


In [10]:
#Baseline:SVD，Surprise，testset only have positive sample.
import surprise as sp
import pandas as pd
import numpy as np
import math
from collections import defaultdict

#1.Loading the dataset and Excluding the outliers
kbdata = pd.read_csv("/data/fjsdata/ctKngBase/kb.csv", sep='|', low_memory=False)
kbdata = kbdata.loc[(kbdata['num']<200)]#seven months, one per day
num_max=kbdata['num'].max()
num_min=kbdata['num'].min()
kbdata['num']=kbdata['num'].apply(lambda x: (x-num_min+1)*1.0/(num_max-num_min+1) )
print ('Dataset shape is:%d rows and %d columns'%(kbdata.shape[0],kbdata.shape[1]))

#2.Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(kbdata[['csr', 'ke', 'num']],reader)
# sampling random trainset and testset, and test set is made of 10% of the ratings.
trainset, testset = sp.model_selection.train_test_split(spdata, test_size=.1)

#3.Training the model and predicting ratings for the testset
algo = sp.SVD()
algo.fit(trainset)
predictions = algo.test(testset)#testset include positive and negtive sample.
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
'''
predPos = algo.test(testset)
testsetNeg = trainset.build_anti_testset(fill=0)    
predNeg = algo.test(testsetNeg)
predictions=predPos.extend(predNeg)#concanate  the pos and the neg
'''

#4.measuring the performance of SVD by precision, recall and  NDCG
#print ('RMSE of testset is:%.8f'%(sp.accuracy.rmse(predictions)))
def calc_dcg(items):
    dcg = 0
    i = 0
    for item in items:
        i += 1
        dcg += (math.pow(2, item) - 1)/ math.log(1 + i, 2)
    return dcg
def index_at_k(predictions, k, threshold=0.1):
   #Return precision and recall at k metrics for each user.
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    ndcgs =dict()
    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        # Number of relevant items
        n_rel = sum((true_r > threshold) for (_, true_r) in user_ratings)
        # Number of recommended items in top k
        n_rec_k = sum((est > threshold) for (est, _) in user_ratings[:k])
        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r > threshold) and (est > threshold)) for (est, true_r) in user_ratings[:k])
        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1
        #true ratings of recommended items in top k
        l_rec_k = [true_r for (_,true_r) in user_ratings[:k]]
        dcg = calc_dcg(l_rec_k)
        #l_rec_k.sort(reverse=True)
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        l_rel_k = [true_r for (_,true_r) in user_ratings[:k]]
        idcg = calc_dcg(l_rel_k)
        ndcgs[uid]=dcg*1.0/idcg 
    return precisions, recalls, ndcgs

print ("%3s%20s%20s%20s" % ('K','Precisions','Recalls','NDCG'))
for k in [5,10,15,20]:#latent factor
    precisions, recalls, ndcgs = index_at_k(predictions, k=k)
    # Precision and recall can then be averaged over all users
    precision = sum(prec for prec in precisions.values()) / len(precisions)
    recall = sum(rec for rec in recalls.values()) / len(recalls)
    ndcg = sum(ndcg for ndcg in ndcgs.values()) / len(ndcgs)
    print ("%3s%20.8f%20.8f%20.8f" % (k, precision, recall, ndcg))

Dataset shape is:2547452 rows and 3 columns
  K          Precisions             Recalls                NDCG
  5          0.73666295          0.66078699          0.72512805
 10          0.73650303          0.66144953          0.77198244
 15          0.73644888          0.66151743          0.79402777
 20          0.73642598          0.66153395          0.80740954


In [29]:
#Baseline:SVD，Surprise，trainset and testset are not splitted samply.
import surprise as sp
import pandas as pd
import numpy as np
import math
from collections import defaultdict

#1.Loading the dataset
trainset = pd.read_csv("/data/fjsdata/ctKngBase/trainset.csv", sep='|', low_memory=False) 
testset = pd.read_csv("/data/fjsdata/ctKngBase/testset.csv", sep='|', low_memory=False)
num_max=max(trainset['num'].max(),testset['num'].max())
num_min=min(trainset['num'].min(),testset['num'].min())
trainset['num']=trainset['num'].apply(lambda x: (x-num_min+1)*1.0/(num_max-num_min+1) )
print ('Trainset shape is:%d rows and %d columns'%(trainset.shape[0],trainset.shape[1]))
testset['num']=testset['num'].apply(lambda x: (x-num_min+1)*1.0/(num_max-num_min+1) )
print ('Testset shape is:%d rows and %d columns'%(testset.shape[0],testset.shape[1]))

#2.Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
trainset = sp.Dataset.load_from_df(trainset[['csr', 'ke', 'num']],reader)
trainset = trainset.build_full_trainset()
#testset = sp.Dataset.load_from_df(testset[['csr', 'ke', 'num']],reader)
testset = np.array(testset).tolist()
#3.Training the model and predicting ratings for the testset
algo = sp.SVD()
algo.fit(trainset)
predictions = algo.test(testset)#testset include positive and negtive sample.

#4.measuring the performance of SVD by precision, recall and  NDCG
#print ('RMSE of testset is:%.8f'%(sp.accuracy.rmse(predictions)))
def calc_dcg(items):
    dcg = 0
    i = 0
    for item in items:
        i += 1
        dcg += (math.pow(2, item) - 1)/ math.log(1 + i, 2)
    return dcg
def index_at_k(predictions, k, threshold=0.1):
   #Return precision and recall at k metrics for each user.
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    ndcgs =dict()
    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        # Number of relevant items
        n_rel = sum((true_r > threshold) for (_, true_r) in user_ratings)
        # Number of recommended items in top k
        n_rec_k = sum((est > threshold) for (est, _) in user_ratings[:k])
        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r > threshold) and (est > threshold)) for (est, true_r) in user_ratings[:k])
        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1
        #true ratings of recommended items in top k
        l_rec_k = [true_r for (_,true_r) in user_ratings[:k]]
        dcg = calc_dcg(l_rec_k)
        #l_rec_k.sort(reverse=True)
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        l_rel_k = [true_r for (_,true_r) in user_ratings[:k]]
        idcg = calc_dcg(l_rel_k)
        ndcgs[uid]=dcg*1.0/idcg 
    return precisions, recalls, ndcgs

print ("%3s%20s%20s%20s" % ('K','Precisions','Recalls','NDCG'))
for k in [5,10,15,20]:#latent factor
    precisions, recalls, ndcgs = index_at_k(predictions, k=k)
    # Precision and recall can then be averaged over all users
    precision = sum(prec for prec in precisions.values()) / len(precisions)
    recall = sum(rec for rec in recalls.values()) / len(recalls)
    ndcg = sum(ndcg for ndcg in ndcgs.values()) / len(ndcgs)
    print ("%3s%20.8f%20.8f%20.8f" % (k, precision, recall, ndcg))

Trainset shape is:2292706 rows and 3 columns
Testset shape is:254746 rows and 3 columns
  K          Precisions             Recalls                NDCG
  5          0.59434487          1.00000000          1.02955465
 10          0.59434487          1.00000000          1.01712075
 15          0.59434487          1.00000000          1.01254969
 20          0.59434487          1.00000000          1.01004584


In [36]:
#Baseline:SVD，Surprise
#Measures: Hit Ratio and NDCG
import surprise as sp
import pandas as pd
import numpy as np
from collections import defaultdict

#1.Loading the dataset and Excluding the outliers
kbdata = pd.read_csv("/data/fjsdata/ctKngBase/kb.csv", sep='|', low_memory=False)
kbdata = kbdata.loc[(kbdata['num']<200)]#7个月，一天一次估算
num_max=kbdata['num'].max()
num_min=kbdata['num'].min()
kbdata['num']=kbdata['num'].apply(lambda x: (x-num_min+1)*1.0/(num_max-num_min+1) )
print ('Dataset shape is:%d rows and %d columns'%(kbdata.shape[0],kbdata.shape[1]))

#2.Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(kbdata[['csr', 'ke', 'num']],reader)
# sampling random trainset and testset, and test set is made of 10% of the ratings.
trainset, testset = sp.model_selection.train_test_split(spdata, test_size=.1)

#3.Training the model and predicting ratings for the testset
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

#4.measuring the performance by precision and recalls
def precision_recall_at_k(predictions, k=10, threshold=0.1):
   #Return precision and recall at k metrics for each user.
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])
        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1
    return precisions, recalls
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=0.1)

# Precision and recall can then be averaged over all users
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

#5.measuring the performance by NDCG and RMSE
RMSE = sp.accuracy.rmse(predictions)

Dataset shape is:2547452 rows and 3 columns
254746
0.7348169845161151
0.6729520766740884


In [34]:
#Baseline:SVD，Surprise
#Measures: Hit Ratio and NDCG
import surprise as sp
import pandas as pd
import numpy as np

#1.Loading the dataset and Excluding the outliers
kbdata = pd.read_csv("/data/fjsdata/ctKngBase/kb.csv", sep='|', low_memory=False)
kbdata = kbdata.loc[(kbdata['num']<200)]#7个月，一天一次估算
num_max=kbdata['num'].max()
num_min=kbdata['num'].min()
kbdata['num']=kbdata['num'].apply(lambda x: (x-num_min+1)*1.0/(num_max-num_min+1) )
print ('Dataset shape is:%d rows and %d columns'%(kbdata.shape[0],kbdata.shape[1]))

#2.Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(kbdata[['csr', 'ke', 'num']],reader)
# sampling random trainset and testset, and test set is made of 10% of the ratings.
trainset, testset = sp.model_selection.train_test_split(spdata, test_size=.1)

#3.Training the model
algo = SVD()
algo.fit(trainset)

#4.predicting ratings for the testset
predictions = algo.test(testset)
print (sp.accuracy.rmse(predictions))

Dataset shape is:2547452 rows and 3 columns
RMSE: 0.0573
0.057304167884117255


In [12]:
#SVD，Surprise
import surprise as sp
import pandas as pd
import numpy as np

#1.Loading the dataset and Excluding the outliers
kbdata = pd.read_csv("/data/fjsdata/ctKngBase/kb.csv", sep='|', low_memory=False)
kbdata = kbdata.loc[(kbdata['num']<200)]#7个月，一天一次估算
num_max=kbdata['num'].max()
num_min=kbdata['num'].min()
kbdata['num']=kbdata['num'].apply(lambda x: (x-num_min+1)*1.0/(num_max-num_min+1) )
print ('Dataset shape is:%d rows and %d columns'%(kbdata.shape[0],kbdata.shape[1]))
testset = kbdata.sample(frac=0.1)
#2.Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(kbdata[['csr', 'ke', 'num']],reader)
trainset = spdata.build_full_trainset()
testset = np.array(testset).tolist()
# sampling random trainset and testset, and test set is made of 10% of the ratings.
#trainset, testset = sp.model_selection.train_test_split(spdata, test_size=.1)
#print(type(testset))

#3.Training the model
algo = sp.SVD()
algo.fit(trainset)

#4.predicting ratings for the testset
predictions = algo.test(testset)
for uid, iid, true_r, est, _ in predictions:
    print (true_r)
    break;
print (sp.accuracy.rmse(predictions))

Dataset shape is:2547452 rows and 3 columns
-0.9849246231155779
RMSE: 0.9981
0.998147312473551


In [ ]:
#Baseline:SVD
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
#@INPUT:
# R     : a matrix to be factorized, dimension N x M
# P     : an initial matrix of dimension N x K
# Q     : an initial matrix of dimension M x K
# K     : the number of latent features
# steps : the maximum number of steps to perform the optimisation
# alpha : the learning rate
#  beta  : the regularization parameter
# @OUTPUT:the final matrices P and Q
def matrix_factorization(R, P, Q, K, steps, alpha=0.0002, beta=0.002):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T
#1.加载训练数据并标准化
kbdata = traindata = pd.read_csv("/data/fjsdata/ctKngBase/kb.csv", sep='|', low_memory=False)
le = LabelEncoder()
kbdata = kbdata.apply(le.fit_transform)
test = kbdata.sample(frac=0.1)#抽样10%比例测试
#2.构建系数矩阵R
nCsr = len(kbdata['csr'].unique()) #用户数
nKe = len(kbdata['ke'].unique()) #知识点数
R = np.zeros((nCsr, nKe))#初始化零矩阵为nCSR*nKe大小
for index, row in kbdata.iterrows(): # 获取每行的值
    R[int(row['csr'])][int(row['ke'])] = 1.0 #正样本
#3.SVD训练和评估
print ("%3s%20s%20s" % ('K','steps','RMSE'))
for K in [50,100,200]:#隐因子
    for steps in [2,500,1000]:#迭代次数
        P = np.random.rand(nCsr,K)
        Q = np.random.rand(nKe,K)
        nP, nQ = matrix_factorization(R, P, Q, K,steps)#SVD分解
        nR = np.dot(nP, nQ.T) #近似矩阵  
        #RMSE评估   
        Y_test = []
        for index, row in test.iterrows(): # 获取每行的值
            nRating=nR[int(row['csr'])][int(row['ke'])] #获取预测值
            Y_test.append(nRating)
        RMSE = mean_squared_error(test['num'],Y_test)
        print ("%3d%20d%20.6f" % (K,steps,RMSE))

  K               steps                RMSE
 50                   2          448.907549
